In [1]:
import math
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import io
import torchtext
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning import loggers as pl_loggers

from pytorch_lightning import seed_everything
seed_everything(42)

42

In [2]:
from sklearn.model_selection import train_test_split

class AlarmDataset(Dataset):
    def __init__(self,data,seq_len,batch_size):
        self.length = len(data)//seq_len # how much data i have         
        self.data = data
        self.seq_len = seq_len
        self.batch_size = batch_size
       
    def __getitem__(self, index: int):
        x = self.data[index*self.seq_len:(index*self.seq_len)+seq_len]
        y = self.data[1+index*self.seq_len:1+(index*self.seq_len)+seq_len]
        return x,y
    
    def __len__(self) -> int:
        return self.length

class MyDataModule(pl.LightningDataModule):
    
    def __init__(self, data_path:str, batch_size:int, seq_len:int):
        super().__init__()
        self.batch_size = batch_size
        self.tokenizer = get_tokenizer('basic_english')
        self.vocab = build_vocab_from_iterator(map(self.tokenizer,iter(io.open(data_path,encoding="utf8"))))
                
        # url = data_path
        # test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
        seqs = None
        with open(data_path) as f:
            seqs = f.readlines()
        print(seqs[:4])
        train, valid = train_test_split(seqs,test_size=0.30,shuffle=False)
        valid, test = train_test_split(valid,test_size=0.30, shuffle=False)

        

        with open("../.data/train.tokens","w") as f:
            for seq in train:
                f.write(seq)
        
        with open("../.data/val.tokens","w") as f:
            for seq in valid:
                f.write(seq)
            
        with open("../.data/test.tokens","w") as f:
            for seq in test:
                f.write(seq)


        
        

        train_data = self.data_process(iter(io.open("../.data/train.tokens", encoding="utf8")))
        val_data = self.data_process(iter(io.open("../.data/val.tokens", encoding="utf8")))
        test_data = self.data_process(iter(io.open("../.data/test.tokens", encoding="utf8")))

    
        self.train_dataset = AlarmDataset(train_data, seq_len,self.batch_size)
        self.valid_dataset = AlarmDataset(val_data,seq_len,self.batch_size)
        self.test_dataset = AlarmDataset(test_data, seq_len,self.batch_size)
    
    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


    def setup(self, stage: None):
        return None

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=False,num_workers=1,drop_last=True, pin_memory=True)
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.valid_dataset, batch_size=self.batch_size, shuffle=False,num_workers=1,drop_last=True, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,num_workers=1,drop_last=True, pin_memory=True)

file_path = '../.data/seqs.tokens'

bsize = 20
seq_len = 35
dm = MyDataModule(file_path,bsize,seq_len)


30551lines [00:00, 54856.64lines/s]
['A17 A17 A17 A75 A17 A57 A17 A17 A17 A98 A99 A56\n', 'A245 A246 A50 A243\n', 'A50 A59 A60 A64 A392 A726 A726 A726 A9 A725 A726 A726 A725 A725 A725 A243 A725\n', 'A746 A17 A266 A563 A204 A613 A367 A1094\n']


In [3]:
class TransformerModel(pl.LightningModule):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5, seq_len=None):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.ntoken = ntoken
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = torch.nn.TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = torch.nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = torch.nn.Linear(ninp, ntoken)
        self.src_mask = self.generate_square_subsequent_mask(seq_len)
        self.seq_len = seq_len 
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src_mask = src_mask.to(self.device)
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        src_mask = src_mask.to(self.device)
      
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=0.0000001)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)

        # print("Training Shape: ", x.size(),y.size())
        
        if x.size(0) != self.seq_len:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)

        loss = F.cross_entropy(y_hat.view(-1, self.ntoken),y)
        self.log('train_loss', loss,on_step=True, prog_bar=True, logger=True)
        self.log("train_ppl",math.exp(loss.item()),on_step=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)

        # print("Validation Shape: ", x.size(),y.size())

        if x.size(0) != self.seq_len:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        # print("> y-hat",y_hat.size())
        loss = F.cross_entropy(y_hat.view(-1, self.ntoken),y)
        self.log('val_loss', loss, on_step=True, prog_bar=True, logger=True)
        self.log("val_ppl",math.exp(loss.item()),on_step=True, prog_bar=True, logger=True)
        return {'val_loss':loss}
    
    def test_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)

        # print("Validation Shape: ", x.size(),y.size())

        if x.size(0) != self.seq_len:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        # print("> y-hat",y_hat.size())
        loss = F.cross_entropy(y_hat.view(-1, self.ntoken),y)
        self.log('test_loss', loss, on_step=True, prog_bar=True, logger=True)
        self.log("test_ppl",math.exp(loss.item()),on_step=True, prog_bar=True, logger=True)
        return {'test_loss':loss}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([d['loss']  for d in outputs]).mean()
        print(f"> Avg Training loss = {avg_loss}")
        
    def validation_epoch_end(self, outputs):
        # print(outputs)
        avg_loss = torch.stack([d['val_loss'] for d in outputs]).mean()
        print(f"> Average Valid Loss = {avg_loss}")
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([d['test_loss'] for d in outputs]).mean()
        print(f"> Average Test Loss = {avg_loss}")
    
    
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [4]:
file_path = '../.data/seqs.tokens'
tb_logger = pl_loggers.TensorBoardLogger('logs/')

bsize = 32
seq_len = 35
dm = MyDataModule(file_path,bsize,seq_len)

ntokens = len(dm.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout,seq_len=seq_len)
trainer = Trainer(precision=16,gpus=1,max_epochs=400,check_val_every_n_epoch=4,deterministic=True, gradient_clip_val=0.5,logger=tb_logger)
trainer.fit(model,dm) # traning and validation

30551lines [00:00, 55218.51lines/s]
['A17 A17 A17 A75 A17 A57 A17 A17 A17 A98 A99 A56\n', 'A245 A246 A50 A243\n', 'A50 A59 A60 A64 A392 A726 A726 A726 A9 A725 A726 A726 A725 A725 A725 A243 A725\n', 'A746 A17 A266 A563 A204 A613 A367 A1094\n']
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name                | Type               | Params
-----------------------------------------------------------
0 | pos_encoder         | PositionalEncoding | 0     
1 | transformer_encoder | TransformerEncoder | 484 K 
2 | encoder             | Embedding          | 138 K 
3 | decoder             | Linear             | 139 K 
-----------------------------------------------------------
762 K     Trainable params
0         Non-trainable params
762 K     Total params


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

> Average Valid Loss = 6.519143104553223


Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

> Average Valid Loss = 2.619338274002075
> Avg Training loss = 1.4847642183303833


Validating: |          | 0/? [00:00<?, ?it/s]

> Average Valid Loss = 2.6048290729522705
> Avg Training loss = 1.4864068031311035


1

In [5]:
trainer.test(datamodule=dm) # testing
# %%

Testing: |          | 0/? [00:00<?, ?it/s]

> Average Test Loss = 2.731191635131836
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(2.7468, device='cuda:0'),
 'test_loss_epoch': tensor(2.7312, device='cuda:0'),
 'test_ppl': 15.592896266917588,
 'test_ppl_epoch': tensor(19.3933),
 'val_loss': tensor(2.1163, device='cuda:0'),
 'val_loss_epoch': tensor(2.6048, device='cuda:0'),
 'val_ppl': 8.30071050411446,
 'val_ppl_epoch': tensor(21.3543)}
--------------------------------------------------------------------------------


[{'val_loss_epoch': 2.6048288345336914,
  'val_ppl_epoch': 21.354297637939453,
  'val_loss': 2.1163411140441895,
  'val_ppl': 8.30071050411446,
  'test_loss_epoch': 2.731191635131836,
  'test_ppl_epoch': 19.39333724975586,
  'test_loss': 2.7468154430389404,
  'test_ppl': 15.592896266917588}]